In [21]:
# setup vars/funcs for scanCore

def dprint(text, end="\r"):
    if DEBUG:
        print(text, end=end)

DEBUG = True

In [22]:
#   creates ip ranges

import random

# create a list of ipv4 ranges
ip_lists = []
for i in range(255):
    for j in range(255):
        ip_lists.append(f"{i}.{j}.0.0/16")
random.shuffle(ip_lists)

# ip_lists = ["172.64.0.0/16","172.65.0.0/16","172.66.0.0/16"] # remove for final vers

In [26]:
#     checks if the ip is runnging a server

from itertools import count
import json
import os
import time

upHosts = []
results = []
start = time.time()

def scan(ip_list):
    try:
        import masscan
        import json

        scanner = masscan.PortScanner()
        scanner.scan(ip_list, ports='25565', arguments='--max-rate 100')
        res = json.loads(scanner.scan_result)
        global results
        results.append(list(res["scan"].keys()))
    except Exception as e:
        print(e)

import threading

print("Starting threads")
threads = []
for host in ip_lists:
    maxActive = 10

    while True: # wait for a thread to finish
        c = 0
        for i in threads:
            if i.is_alive():
                c += 1
        if c <= maxActive:
            break
        
        time.sleep(0.1)
            
    
    t = threading.Thread(target=scan, args=(host,))
    t.start()
    threads.append(t)

    c = 0
    for result in results:
        c += len(result)
    print(f"Started thread for {host} | Total Threads: {len(threads)} | Total Found {c}")

# monitor threads
while True:
    time.sleep(0.1)
    if not any(t.is_alive() for t in threads):
        break
c = 0
for result in results:
    c += len(result)

print(f"found {c} hosts | average time/list {round(((time.time()-start)/(len(ip_lists))),2)} {' '*40}")

Starting threads
Started thread for 53.95.0.0/16 | Total Threads: 1 | Total Found 0
Started thread for 2.26.0.0/16 | Total Threads: 2 | Total Found 0
Started thread for 32.100.0.0/16 | Total Threads: 3 | Total Found 0
Started thread for 22.32.0.0/16 | Total Threads: 4 | Total Found 0
Started thread for 83.138.0.0/16 | Total Threads: 5 | Total Found 0
Started thread for 252.215.0.0/16 | Total Threads: 6 | Total Found 0
Started thread for 124.154.0.0/16 | Total Threads: 7 | Total Found 0
Started thread for 14.35.0.0/16 | Total Threads: 8 | Total Found 0
Started thread for 219.199.0.0/16 | Total Threads: 9 | Total Found 0
Started thread for 58.126.0.0/16 | Total Threads: 10 | Total Found 0
Started thread for 107.151.0.0/16 | Total Threads: 11 | Total Found 0
Started thread for 16.119.0.0/16 | Total Threads: 12 | Total Found 0
Started thread for 30.3.0.0/16 | Total Threads: 13 | Total Found 0
Started thread for 150.211.0.0/16 | Total Threads: 14 | Total Found 0
Started thread for 88.116.0.

KeyboardInterrupt: 

In [27]:
#     checks if the ip is runnging a minecraft server

import time

start_time = time.time()
res = []
lst = []
for result in results:
    for host in result:
        lst.append(host)

def check(host):
    try:
        import mcstatus
        print("\r",end="")
        server = mcstatus.JavaServer.lookup(host)
        status = server.status()
        description = status.description
        #remove color codes and whitespace
        description = description.replace("§0","").replace("§1","").replace("§2","").replace("§3","").replace("§4","").replace("§5","").replace("§6","").replace("§7","").replace("§8","").replace("§9","").replace("§a","").replace("§b","").replace("§c","").replace("§d","").replace("§e","").replace("§f","").replace("§k","").replace("§l","").replace("§m","").replace("§n","").replace("§o","").replace("§r","").replace(" ","")
        dprint(f"{host} ({description}): has {status.players.online} players and replied in {round(status.latency,2)} ms\n")

        res.append(host)
    except Exception as e:
        pass

import threading

print("Starting threads")
threads = []
for host in lst:
    thread = threading.Thread(target=check, args=(host,))
    thread.start()
    threads.append(thread)

# monitor threads
while True:
    time.sleep(0.1)
    if not any(t.is_alive() for t in threads):
        break


Starting threads
142.114.244.115 (AMinecraftServer): has 0 players and replied in 2.34 ms
72.137.167.250 (AMinecraftServer): has 0 players and replied in 1.95 ms
72.137.219.2 (MinecraftServer): has 0 players and replied in 2.13 ms
142.114.34.95 (AMinecraftServer): has 1 players and replied in 2.09 ms
142.114.93.114 (AMinecraftServer): has 0 players and replied in 2.32 ms
142.114.139.210 (Naughtygamerswillenduponthefence.): has 0 players and replied in 2.11 ms
72.5.102.176 (NFOservers.com:Newserver): has 0 players and replied in 1.01 ms
72.5.54.184 (PALSPALSPALSPALS): has 0 players and replied in 1.39 ms
72.137.75.50 (AlltheMods7): has 0 players and replied in 3.5 ms
34.214.9.255 (KondayIsland): has 0 players and replied in 0.02 ms
34.214.244.42 (Boredoflife?HaveaTekxitPi!): has 0 players and replied in 0.73 ms
175.178.100.218 (AMinecraftServer): has 0 players and replied in 0.84 ms
175.178.230.109 (AMinecraftServer): has 0 players and replied in 0.34 ms
175.178.26.76 (AMinecraftServer)

In [28]:
#   Deal with the data
print(f"found {len(res)} servers")

found 229 servers in 424.7 seconds
